In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import re
import dateparser 

Merge speeches dataframe with presidents dataframe - since there are some slight variations in the president names between these two tables, we will use a fuzzy merge technique

In [10]:
speeches = pd.read_csv("./data/presidential_speeches.csv")
parties = pd.read_csv("./data/presidential_party_and_term.csv")

# change the column name of the presidents dataframe to match the speeches dataframe
# In the presidents dataframe, the column name is 'president'. In the speeches dataframe, the column name is 'name'
parties = parties.rename(columns={'president':'name'})

In [11]:
speeches.head()

,name,title,date,about,speech
0,Joe Biden,"February 21, 2023: Remarks on the One-Year Ann...",2023-02-21,"Speaking at the Royal Castle in Warsaw, Poland...","THE PRESIDENT: Hello, Poland! One of our great..."
1,Joe Biden,"February 7, 2023: State of the Union Address",2023-02-07,"In his State of the Union Address, President J...",Mr. Speaker. Madam Vice President. Our Firs...
2,Joe Biden,"September 21, 2022: Speech before the 77th Ses...",2022-09-21,President Joe Biden addresses the 77th session...,"Thank you. Mr. President, Mr. Secretary-Gene..."
3,Joe Biden,"September 1, 2022: Remarks on the Continued Ba...",2022-09-01,"President Joe Biden speaks in Philadelphia, Pe...","THE PRESIDENT: My fellow Americans, please, if..."
4,Joe Biden,"May 24, 2022: Remarks on School Shooting in Uv...",2022-05-24,President Biden makes an impassioned plea to s...,"Good evening, fellow Americans. I had hoped, w..."


In [12]:
parties.head()

,name,birthplace,political party,from,to
0,George Washington,Va.,Federalist,1789,1797
1,John Adams,Mass.,Federalist,1797,1801
2,Thomas Jefferson,Va.,Democratic-Republican,1801,1809
3,James Madison,Va.,Democratic-Republican,1809,1817
4,James Monroe,Va.,Democratic-Republican,1817,1825


In [13]:
import difflib # this will provide us with a 'fuzzy' match between the presidential names found in each table

#convert name in party to name it most closely matches in speeches
parties['name'] = parties['name'].apply(lambda x: difflib.get_close_matches(x, speeches['name'])[0])

In [14]:
# merge the DataFrames into one
merged = speeches.merge(parties)

# view final DataFrame
merged

,name,title,date,about,speech,birthplace,political party,from,to
0,Joe Biden,"February 21, 2023: Remarks on the One-Year Ann...",2023-02-21,"Speaking at the Royal Castle in Warsaw, Poland...","THE PRESIDENT: Hello, Poland! One of our great...",Pa.,Democratic,2021,2025
1,Joe Biden,"February 7, 2023: State of the Union Address",2023-02-07,"In his State of the Union Address, President J...",Mr. Speaker. Madam Vice President. Our Firs...,Pa.,Democratic,2021,2025
2,Joe Biden,"September 21, 2022: Speech before the 77th Ses...",2022-09-21,President Joe Biden addresses the 77th session...,"Thank you. Mr. President, Mr. Secretary-Gene...",Pa.,Democratic,2021,2025
3,Joe Biden,"September 1, 2022: Remarks on the Continued Ba...",2022-09-01,"President Joe Biden speaks in Philadelphia, Pe...","THE PRESIDENT: My fellow Americans, please, if...",Pa.,Democratic,2021,2025
4,Joe Biden,"May 24, 2022: Remarks on School Shooting in Uv...",2022-05-24,President Biden makes an impassioned plea to s...,"Good evening, fellow Americans. I had hoped, w...",Pa.,Democratic,2021,2025
...,...,...,...,...,...,...,...,...,...
1086,George Washington,"December 29, 1790: Talk to the Chiefs and Coun...",1790-12-29,The President reassures the Seneca Nation that...,"I the President of the United States, by my ...",Va.,Federalist,1789,1797
1087,George Washington,"December 8, 1790: Second Annual Message to Con...",1790-12-08,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of R...,Va.,Federalist,1789,1797
1088,George Washington,"January 8, 1790: First Annual Message to Congress",1790-01-08,"In a wide-ranging speech, President Washington...",Fellow Citizens of the Senate and House of Re...,Va.,Federalist,1789,1797
1089,George Washington,"October 3, 1789: Thanksgiving Proclamation",1789-10-03,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackn...,Va.,Federalist,1789,1797


In [15]:
merged.to_csv("./data/presidential_speeches_merged.csv",index=False)

In [16]:
merged['political party'].value_counts()

political party
Democratic               497
Republican               439
Democratic-Republican     56
Democratic (Union)        31
Whig                      30
Federalist                30
National Republican        8
Name: count, dtype: int64